## Week Five Class One: Pizza OM Book problem

Begin by importing pandas and pyomo.environ using the aliases 'pd' and 'pe'

In [1]:
import pandas as pd
import pyomo.environ as pe

## Import Data From Excel File

Import the data from the excel file `w05-c01-covering-allocation.xlsx`. Make sure to import from the 'Pizza' sheet.

In [2]:
raw_data = pd.read_excel("w05-c01-covering-allocation.xlsx", sheet_name = "Pizza") # Fill in the information

## Define the Decision Variable Indexes (Names)

In [3]:
DV_indexes = ['plain', 'meat', 'veggie','supreme']

## Define a coef DataFrame

Create a dataframe 'coef' to hold the coefficients of the objective function (revenue) and the LHS of the ingredient constraints. 

In [4]:
raw_data

,Decision variables (pizza types),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,plain,meat,veggie,supreme,NaN,NaN,NaN,NaN,NaN
1,NaN,0,10,0,20,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Objective (maximize revenue),NaN,NaN,NaN,NaN,NaN,value,NaN,NaN,NaN
4,NaN,8,10,12,15,NaN,400,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Constraints,NaN,NaN,NaN,NaN,NaN,LHS,NaN,RHS,slack
7,dough,5,5,5,5,NaN,150,<=,200,50
8,sauce,3,3,3,3,NaN,90,<=,90,0
9,cheese,4,3,3,4,NaN,110,<=,120,10


In [5]:
coef = pd.DataFrame(raw_data.iloc[[4, 7, 8, 9, 10, 11], 1:5])  #Fill in the missing indexes***
coef.index = ['revenue', 'dough', 'sauce', 'cheese', 'meat', 'veggies']
coef.columns = DV_indexes
coef

,plain,meat,veggie,supreme
revenue,8,10,12,15
dough,5,5,5,5
sauce,3,3,3,3
cheese,4,3,3,4
meat,0,3,0,2
veggies,0,0,3,2


## Define a rhs DataFrame

Next create a dataframe 'rhs' to hold the RHS of the constraints into their own dataframe. 

In [6]:
rhs = pd.DataFrame(raw_data.iloc[7:12, 8]) #Fill in the missing indexes***
rhs.index = coef.index[1:6]
rhs.columns = ['rhs']
rhs

,rhs
dough,200
sauce,90
cheese,120
meat,75
veggies,40


## Define the pyomo model

First instantiate a ConcreteModel and store it in the variable `model`.

In [7]:
model = pe.ConcreteModel()

## Define the pyomo Decision Variables

Create a pyomo variable named `x` with domain of nonnegative of real numbers. Use the DV_indexes as the index for the variables. View the variables with `.pprint()`.

In [8]:
model.x = pe.Var(DV_indexes, domain = pe.NonNegativeReals) #Fill in the missing decision variable information
model.x.pprint()

x : Size=4, Index=x_index
    Key     : Lower : Value : Upper : Fixed : Stale : Domain
       meat :     0 :  None :  None : False :  True : NonNegativeReals
      plain :     0 :  None :  None : False :  True : NonNegativeReals
    supreme :     0 :  None :  None : False :  True : NonNegativeReals
     veggie :     0 :  None :  None : False :  True : NonNegativeReals


## Define the Objective Function

Next define the objective function `obj` using the data in the `coef` dataframe.  This should maximize revenue. Double check that it matches the correct formula with `.pprint`.

In [9]:
model.obj = pe.Objective(expr = sum([coef.loc['revenue', i]*model.x[i] for i in DV_indexes]),  
                         sense = pe.maximize)
model.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : maximize : 8*x[plain] + 10*x[meat] + 12*x[veggie] + 15*x[supreme]


Next we need to define the five constraints using the data in the `coef` and `rhs` dataframes. 

In [10]:
model.cons_dough = pe.Constraint(expr=sum([coef.loc['dough', i]*model.x[i] for i in DV_indexes]) <= rhs.loc['dough','rhs'])
model.cons_sauce = pe.Constraint(expr=sum([coef.loc['sauce', i]*model.x[i] for i in DV_indexes]) <= rhs.loc['sauce','rhs'])
model.cons_cheese = pe.Constraint(expr=sum([coef.loc['cheese', i]*model.x[i] for i in DV_indexes]) <= rhs.loc['cheese','rhs'])
model.cons_meat = pe.Constraint(expr=sum([coef.loc['meat', i]*model.x[i] for i in DV_indexes]) <= rhs.loc['meat','rhs'])
model.cons_veggies = pe.Constraint(expr=sum([coef.loc['veggies', i]*model.x[i] for i in DV_indexes]) <= rhs.loc['veggies','rhs'])

Now before we do the final solve, display the entire model with `.pprint`.

In [11]:
model.pprint()

1 Set Declarations
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'plain', 'meat', 'veggie', 'supreme'}

1 Var Declarations
    x : Size=4, Index=x_index
        Key     : Lower : Value : Upper : Fixed : Stale : Domain
           meat :     0 :  None :  None : False :  True : NonNegativeReals
          plain :     0 :  None :  None : False :  True : NonNegativeReals
        supreme :     0 :  None :  None : False :  True : NonNegativeReals
         veggie :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 8*x[plain] + 10*x[meat] + 12*x[veggie] + 15*x[supreme]

5 Constraint Declarations
    cons_cheese : Size=1, Index=None, Active=True
        Key  : Lower : Body                                                : Upper : Active
        None :  

## Solve the model

In [12]:
opt = pe.SolverFactory('glpk')
success = opt.solve(model, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /var/folders/p8/y_pk7h153tld8fgnq6fz274r0000gn/T/tmpi1psh3d2.glpk.raw
 --wglp /var/folders/p8/y_pk7h153tld8fgnq6fz274r0000gn/T/tmp_17b8wu_.glpk.glp
 --cpxlp /var/folders/p8/y_pk7h153tld8fgnq6fz274r0000gn/T/tmp52tvt5m2.pyomo.lp
Reading problem data from '/var/folders/p8/y_pk7h153tld8fgnq6fz274r0000gn/T/tmp52tvt5m2.pyomo.lp'...
6 rows, 5 columns, 17 non-zeros
51 lines were read
Writing problem data to '/var/folders/p8/y_pk7h153tld8fgnq6fz274r0000gn/T/tmp_17b8wu_.glpk.glp'...
41 lines were written
GLPK Simplex Optimizer, v4.65
6 rows, 5 columns, 17 non-zeros
Preprocessing...
5 rows, 4 columns, 16 non-zeros
Scaling...
 A: min|aij| =  2.000e+00  max|aij| =  5.000e+00  ratio =  2.500e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 5
*     0: obj =  -0.000000000e+00 inf =   0.000e+00 (4)
*     2: obj =   4.000000000e+02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTI

In [13]:
print(success)


Problem: 
- Name: unknown
  Lower bound: 400.0
  Upper bound: 400.0
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 5
  Number of nonzeros: 17
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.016948938369750977
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



## View the Final Model Obj Function and Dec Variables

Now display the parts of the model solution. First, by outputting the objective function and Decision variables for the optimal solution.

In [14]:
obj_val = model.obj.expr()
print(f'optimal objective value maximum profit = ${obj_val:.2f}')

DV = []  # create an empty list to store decision variables
for index in DV_indexes:
    DV.append(round(model.x[index].value, 3))
pd.DataFrame({'DV':DV_indexes,
             'Value':DV})

optimal objective value maximum profit = $400.00


,DV,Value
0,plain,0.0
1,meat,10.0
2,veggie,0.0
3,supreme,20.0


## View the Slack

Print the slack in each constraint.  Do these match the Excel output?

In [15]:
print(f'Dough slack = {model.cons_dough.slack()}')
print(f'Cheese slack = {model.cons_cheese.slack()}')
print(f'Veggie slack = {model.cons_veggies.slack()}')
print(f'Sauce slack = {model.cons_sauce.slack()}')
print(f'Meat slack = {model.cons_meat.slack()}')

# Fill in the rest of the constraints slack



Dough slack = 50.0
Cheese slack = 10.0
Veggie slack = 0.0
Sauce slack = 0.0
Meat slack = 5.0


## View the final LHS values

Also examine the final values for the LHS for each constraint. The "Body" is the final value. Do these match the Excel output?

In [16]:
model.cons_dough.display()
model.cons_cheese.display()
model.cons_veggies.display()
model.cons_sauce.display()
model.cons_meat.display()

cons_dough : Size=1
    Key  : Lower : Body  : Upper
    None :  None : 150.0 : 200.0
cons_cheese : Size=1
    Key  : Lower : Body  : Upper
    None :  None : 110.0 : 120.0
cons_veggies : Size=1
    Key  : Lower : Body : Upper
    None :  None : 40.0 :  40.0
cons_sauce : Size=1
    Key  : Lower : Body : Upper
    None :  None : 90.0 :  90.0
cons_meat : Size=1
    Key  : Lower : Body : Upper
    None :  None : 70.0 :  75.0
